In [1]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [2]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

In [3]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df.head())

len(df): 34
  date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12 sym_freq_11 sym_freq_10 sym_freq_9       sym_freq_8             sym_freq_7                     sym_freq_6                     sym_freq_5                     sym_freq_4                   sym_freq_3 sym_freq_2
0        2023-04-06                 120      [30, 60, 120]          []          []          []          []          []          []         []          ['SHV']               ['FTSM']                ['ELF', 'SGEN']       ['META', 'NVDA', 'PDFS']  ['ACLS', 'AMPH', 'AU', 'GB...  ['AJRD', 'ANET', 'BTC-USD']         []
1        2023-04-06                 120  [15, 30, 60, 120]          []          []          []          []          []          []         []          ['SHV']         ['AU', 'FTSM']  ['ELF', 'NVO', 'SGEN', 'SNY']       ['META', 'NVDA', 'PDFS']  ['ACLS', 'AMPH', 'CNK', 'G...                     ['AJRD']         []
2        2023-04-05                 120    

In [4]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  _date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == _date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if _date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = _date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-04-06    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['SHV']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : []
5                   [15, 30, 60, 120]    sym_freq_8 : ['SHV']

6     2023-04-05    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : ['SHV']
8                   [30, 60, 120]        sym_freq_8 : []
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : ['SHV']
11                  [15, 30, 60, 120]    sym_freq_8 : []

12    2023-04-04    [30, 60, 120]        sym

In [5]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [6]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-04-06 [30, 60, 120] sym_freq_12 []
1 2023-04-06 [30, 60, 120] sym_freq_9 []
2 2023-04-06 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-06', [30, 60, 120], 'sym_freq_8', ['SHV']]

3 2023-04-06 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-06 [15, 30, 60, 120] sym_freq_9 []
5 2023-04-06 [15, 30, 60, 120] sym_freq_8 ['SHV']
6 2023-04-05 [30, 60, 120] sym_freq_12 []
7 2023-04-05 [30, 60, 120] sym_freq_9 ['SHV']
added row to df_model_top_picks:
['2023-04-05', [30, 60, 120], 'sym_freq_9', ['SHV']]

8 2023-04-05 [30, 60, 120] sym_freq_8 []
9 2023-04-05 [15, 30, 60, 120] sym_freq_12 []
10 2023-04-05 [15, 30, 60, 120] sym_freq_9 ['SHV']
11 2023-04-05 [15, 30, 60, 120] sym_freq_8 []
12 2023-04-04 [30, 60, 120] sym_freq_12 []
13 2023-04-04 [30, 60, 120] sym_freq_9 []
14 2023-04-04 [30, 60, 120] sym_freq_8 ['NVDA', 'SHV']
added row to df_model_top_picks:
['2023-04-04', [30, 60, 120], 'sym_freq_8', ['NVDA', 'SHV']]

15 2023-04-04 [15, 30, 60, 120] sym_freq_12 []
16

In [7]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates(keep='last')
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 22
df_model_top_picks:
          date  days_lookback   syms_freq          symbols
0   2023-04-06  [30, 60, 120]  sym_freq_8          ['SHV']
1   2023-04-05  [30, 60, 120]  sym_freq_9          ['SHV']
2   2023-04-04  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
3   2023-04-03  [30, 60, 120]  sym_freq_8   ['GE', 'NVDA']
4   2023-03-31  [30, 60, 120]  sym_freq_8         ['NVDA']
5   2023-03-30  [30, 60, 120]  sym_freq_9          ['SHV']
6   2023-03-30  [30, 60, 120]  sym_freq_8         ['NVDA']
7   2023-03-29  [30, 60, 120]  sym_freq_9          ['SHV']
8   2023-03-28  [30, 60, 120]  sym_freq_9          ['SHV']
9   2023-03-27  [30, 60, 120]  sym_freq_9          ['SHV']
10  2023-03-27  [30, 60, 120]  sym_freq_8           ['GE']
11  2023-03-24  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
12  2023-03-23  [30, 60, 120]  sym_freq_9          ['SHV']
13  2023-03-22  [30, 60, 120]  sym_freq_9          ['SHV']
14  2023-03-22  [30, 60, 120]  sym_freq_8         ['FTSM']
15  2023

In [8]:
df_mp = df_model_top_picks.copy()
df_close = pickle_load(path_data_dump, 'df_close_clean')

In [9]:
import pandas_market_calendars as mcal
import datetime
today = datetime.date.today()
yr_10 = datetime.timedelta(days = 365*10)

nyse = mcal.get_calendar('NYSE')
dates_NYSE_past_10yr = nyse.valid_days(start_date=today - yr_10, end_date=today)

In [10]:
sorted_df_close_idx = sorted(df_close.index)
print(f'len(sorted_df_close_idx: {len(sorted_df_close_idx)}')
print(f'sorted_df_close_idx[:3]: {sorted_df_close_idx[:3]}')
print(f'sorted_df_close_idx[-3:]: {sorted_df_close_idx[-3:]}\n')

print(f'len(dates_NYSE_past_10yr: {len(dates_NYSE_past_10yr)}')
print(f'dates_NYSE_past_10yr: {dates_NYSE_past_10yr[:3]}')
print(f'dates_NYSE_past_10yr: {dates_NYSE_past_10yr[-3:]}')

len(sorted_df_close_idx: 1500
sorted_df_close_idx[:3]: [Timestamp('2017-04-24 00:00:00'), Timestamp('2017-04-25 00:00:00'), Timestamp('2017-04-26 00:00:00')]
sorted_df_close_idx[-3:]: [Timestamp('2023-04-04 00:00:00'), Timestamp('2023-04-05 00:00:00'), Timestamp('2023-04-06 00:00:00')]

len(dates_NYSE_past_10yr: 2516
dates_NYSE_past_10yr: DatetimeIndex(['2013-04-11 00:00:00+00:00', '2013-04-12 00:00:00+00:00', '2013-04-15 00:00:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
dates_NYSE_past_10yr: DatetimeIndex(['2023-04-04 00:00:00+00:00', '2023-04-05 00:00:00+00:00', '2023-04-06 00:00:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)


In [11]:
def get_next_NYSE_date(my_date, dates_NYSE_past_10yr):

  if my_date in dates_NYSE_past_10yr:
    idx_date = dates_NYSE_past_10yr.get_loc(my_date)

    if idx_date >= len(dates_NYSE_past_10yr) - 1:
      print(f'Next NYSE trading date after {my_date} is not a trading date in dates_NYSE_past_10yr')
      print(f'dates_NYSE_past_10yr:\n{dates_NYSE_past_10yr}')

    else:
      date_next = dates_NYSE_past_10yr[idx_date + 1].strftime('%Y-%m-%d')  

  else:
    msg_stop_date_NYSE = f'ERROR: my_date: {my_date} is not a trading date in dates_NYSE_past_10yr\ndates_NYSE_past_10yr:\n{dates_NYSE_past_10yr}'
    raise SystemExit(msg_stop_date_NYSE )

  return date_next  

In [12]:
def get_symbols_close(my_date, df_close):
  if my_date in df_close.index:  
    iloc_my_date = df_close.index.get_loc(my_date)
    print(f'len(df_close): {len(df_close)}, my_date: {my_date}, iloc_my_date: {iloc_my_date}')      
    symbols_close = df_close.iloc[iloc_my_date]    
  else:
    msg_stop = f'ERROR: my_date: {my_date} is not a date in df_close.index\ndf_close.index:\n{df_c.index}'
    raise SystemExit(msg_stop)
  return symbols_close


In [13]:
date_trade = '2017-04-24'
date_next_trade = get_next_NYSE_date(date_trade, dates_NYSE_past_10yr)
date_next_trade

'2017-04-25'

In [14]:
symbols_close = get_symbols_close(date_next_trade, df_close)
print(f'type(symbols_close): {type(symbols_close)}')
symbols_close

len(df_close): 1500, my_date: 2017-04-25, iloc_my_date: 1
type(symbols_close): <class 'pandas.core.series.Series'>


A        52.352097
AA       36.077797
AAL      45.140968
AAP     136.706787
AAPL     33.909439
           ...    
ZION     34.756897
ZTO      12.349427
ZTS      53.150013
ZUMZ     18.350000
ZWS      23.325800
Name: 2017-04-25 00:00:00, Length: 1319, dtype: float64

In [18]:
import numpy as np
from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)
portf_target = 1000  # ideal value of initial portfolio amount 
for _date, _syms in z_date_syms:
    l_syms = literal_eval(_syms)  # convert list stored as str back to list
    print(_date, l_syms)
    _next_date = get_next_NYSE_date(_date, dates_NYSE_past_10yr)
    print(_date, _next_date, l_syms)

    # ar_price = df_close.loc[_date][l_syms].values
    # sym_cnt = len(l_syms)
    # # print(_date, l_syms, ar_price, sym_cnt)
    # amt_per_sym = portf_target / sym_cnt
    # ar_sym_share = np.floor(amt_per_sym / ar_price)
    # ar_sym_amt = ar_price * ar_sym_share
    # portf_amt = sum(ar_sym_amt)
    # print(_date, l_syms, ar_price, sym_cnt, ar_sym_share, ar_sym_amt, portf_amt)    

2023-04-06 ['SHV']
Next NYSE trading date after 2023-04-06 is not a trading date in dates_NYSE_past_10yr
dates_NYSE_past_10yr:
DatetimeIndex(['2013-04-11 00:00:00+00:00', '2013-04-12 00:00:00+00:00', '2013-04-15 00:00:00+00:00', '2013-04-16 00:00:00+00:00', '2013-04-17 00:00:00+00:00', '2013-04-18 00:00:00+00:00', '2013-04-19 00:00:00+00:00', '2013-04-22 00:00:00+00:00', '2013-04-23 00:00:00+00:00', '2013-04-24 00:00:00+00:00',
               ...
               '2023-03-24 00:00:00+00:00', '2023-03-27 00:00:00+00:00', '2023-03-28 00:00:00+00:00', '2023-03-29 00:00:00+00:00', '2023-03-30 00:00:00+00:00', '2023-03-31 00:00:00+00:00', '2023-04-03 00:00:00+00:00', '2023-04-04 00:00:00+00:00', '2023-04-05 00:00:00+00:00', '2023-04-06 00:00:00+00:00'], dtype='datetime64[ns, UTC]', length=2516, freq=None)


UnboundLocalError: local variable 'date_next' referenced before assignment

In [15]:
import numpy as np
from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)
portf_target = 1000  # ideal value of initial portfolio amount 
for _date, _syms in z_date_syms:
    l_syms = literal_eval(_syms)  # convert list stored as str back to list
    # print(_date, _syms, type(_date), type(_syms))
    ar_price = df_close.loc[_date][l_syms].values
    sym_cnt = len(l_syms)
    # print(_date, l_syms, ar_price, sym_cnt)
    amt_per_sym = portf_target / sym_cnt
    ar_sym_share = np.floor(amt_per_sym / ar_price)
    ar_sym_amt = ar_price * ar_sym_share
    portf_amt = sum(ar_sym_amt)
    print(_date, l_syms, ar_price, sym_cnt, ar_sym_share, ar_sym_amt, portf_amt)    

2023-04-06 ['SHV'] [110.19999695] 1 [9.] [991.79997253] 991.7999725341797
2023-04-05 ['SHV'] [110.19999695] 1 [9.] [991.79997253] 991.7999725341797
2023-04-04 ['NVDA', 'SHV'] [274.52999878 110.13999939] 2 [1. 4.] [274.52999878 440.55999756] 715.0899963378906
2023-04-03 ['GE', 'NVDA'] [ 96.91999817 279.6499939 ] 2 [5. 1.] [484.59999084 279.6499939 ] 764.2499847412109
2023-03-31 ['NVDA'] [277.76998901] 1 [3.] [833.30996704] 833.3099670410156
2023-03-30 ['SHV'] [110.09902954] 1 [9.] [990.89126587] 990.8912658691406
2023-03-30 ['NVDA'] [273.82998657] 1 [3.] [821.48995972] 821.4899597167969
2023-03-29 ['SHV'] [110.07910156] 1 [9.] [990.71191406] 990.7119140625
2023-03-28 ['SHV'] [110.06913757] 1 [9.] [990.62223816] 990.6222381591797
2023-03-27 ['SHV'] [110.05917358] 1 [9.] [990.53256226] 990.5325622558594
2023-03-27 ['GE'] [93.30999756] 1 [10.] [933.09997559] 933.0999755859375
2023-03-24 ['NVDA', 'SHV'] [267.79000854 110.05917358] 2 [1. 4.] [267.79000854 440.23669434] 708.0267028808594
2023